# Real Bogus Experiment

We used two subtraction techniques

Zackay's and Bramich's DIA algorithms.
Both were applied on the same new--ref pair of images and the same source identification algorithm.
Both dataset are comparable... 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
sns.set_context(font_scale=1.2)

In [ ]:
from sqlalchemy import create_engine
import pandas as pd

In [ ]:
import os
plot_dir = os.path.abspath('./plots/.')
if not os.path.isdir(plot_dir):
    os.makedirs(plot_dir)

In [ ]:
engine = create_engine('sqlite:///../RBpipeline/rbogus-dev.db')

Asking to the database for the undetected objects (False Negatives)

In [ ]:
simulated = pd.read_sql_query("""SELECT * FROM Simulated""", engine)

In [ ]:
und_z = pd.read_sql_query("""SELECT S.x, S.y, S.app_mag, S.image_id, S.r_scales, S.gx_mag, U.simulated_id 
                             FROM Simulated S INNER JOIN Undetected U
                             ON S.id==U.simulated_id""", 
                          engine)

In [ ]:
und_b = pd.read_sql_query("""SELECT S.x, S.y, S.app_mag, S.image_id, S.r_scales, S.gx_mag, U.simulated_id 
                             FROM Simulated S INNER JOIN UndetectedOIS U
                             ON S.id==U.simulated_id""", 
                          engine)

Querying the database to obtain the tables related to the detections.

In [ ]:
dt_zps = pd.merge(pd.read_sql_table('Detected', engine),
                  pd.read_sql_query("""SELECT 
                                        Detected.id,
                                        Simulated.app_mag as sim_mag,
                                        Simulated.id as sim_id,
                                        Simulated.r_scales as r_scales,
                                        Simulated.gx_mag as gx_mag 
                                    FROM Detected
                                        LEFT JOIN Reals
                                            ON Detected.id==Reals.detected_id
                                        LEFT JOIN Simulated
                                            ON Simulated.id==Reals.simulated_id""", engine),
                                      on='id', suffixes=('',''))

dt_ois = pd.merge(pd.read_sql_table('DetectedOIS', engine),
                  pd.read_sql_query("""SELECT 
                                        DetectedOIS.id,
                                        Simulated.app_mag as sim_mag,
                                        Simulated.id as sim_id,
                                        Simulated.r_scales as r_scales,
                                        Simulated.gx_mag as gx_mag  
                                    FROM DetectedOIS
                                        LEFT JOIN RealsOIS
                                            ON DetectedOIS.id==RealsOIS.detected_id
                                        LEFT JOIN Simulated
                                            ON Simulated.id==RealsOIS.simulated_id""", engine),
                                      on='id', suffixes=('',''))

In [ ]:
# dt_ois.IS_REAL = dt_ois.IS_REAL.astype(int)
# dt_zps.IS_REAL = dt_zps.IS_REAL.astype(int)

### Some plots of simulated objects

In [ ]:
plt.figure(figsize=(4,3))
plt.hist(simulated['app_mag'], cumulative=True)
plt.xlabel(r'$mag$')
plt.ylabel(r'$\int_{-\infty}^{mag}\phi(m\prime)dm\prime$')
plt.savefig(os.path.join(plot_dir, 'lum_fun_simulated.png'), dpi=700)

### Analyzing the fraction of type of objects for each dataset

In [ ]:
ois = [sum(dt_ois.IS_REAL), len(und_b), len(dt_ois.IS_REAL)-sum(dt_ois.IS_REAL)]
zps = [sum(dt_zps.IS_REAL), len(und_z), len(dt_zps.IS_REAL)-sum(dt_zps.IS_REAL)]
df2 = pd.DataFrame([zps, ois], columns=['Real', 'False Neg', 'Bogus'])

In [ ]:
print df2

In [ ]:
plt.figure(figsize=(4, 3))
df2.plot.barh(stacked=True)
plt.title('Fraction of classes')
plt.xscale('linear')
plt.xlim(0, 140000)
plt.yticks([0,1], ['Zackay', 'Bramich'])

plt.legend(loc='lower right')
plt.savefig(os.path.join(plot_dir, 'fractions_classes.svg'), dpi=700)

## Magnitude offset

We need to calculate the magnitude offset to understand how to compare perfomances.

In [ ]:
cx_zps = pd.read_sql_query(
"""SELECT 
  Detected.MAG_ISO, 
  Detected.MAG_AUTO, 
  Detected.MAG_APER,
  Simulated.app_mag as sim_mag,
  Simulated.id as sim_id 
FROM Detected
  INNER JOIN Reals
    ON Detected.id==Reals.detected_id
  INNER JOIN Simulated
    ON Simulated.id==Reals.simulated_id""", engine)

In [ ]:
cx_ois = pd.read_sql_query(
"""SELECT 
  DetectedOIS.MAG_ISO,
  DetectedOIS.MAG_AUTO,
  DetectedOIS.MAG_APER,
  Simulated.app_mag as sim_mag,
  Simulated.id as sim_id 
FROM DetectedOIS
  INNER JOIN RealsOIS
    ON DetectedOIS.id==RealsOIS.detected_id
  INNER JOIN Simulated
    ON Simulated.id==RealsOIS.simulated_id""", engine)

In [ ]:
detections = pd.merge(cx_ois, cx_zps, on='sim_id', suffixes=('_ois','_zps'))

In [ ]:
detections['MAG_OFFSET_zps'] = detections['sim_mag_zps'] - detections['MAG_ISO_zps']
detections['MAG_OFFSET_ois'] = detections['sim_mag_ois'] - detections['MAG_ISO_ois']

In [ ]:
plt.figure(figsize=(6,3))
plt.subplot(121)
plt.hist(detections['MAG_OFFSET_ois'], log=True, color='red')
plt.title('Bramich')
plt.xlabel(r'$\Delta$ Mag')
plt.subplot(122)
plt.hist(detections['MAG_OFFSET_zps'], log=True, color='blue')
plt.xlabel(r'$\Delta$ Mag')
plt.title('Zackay')
#plt.show()

plt.savefig(os.path.join(plot_dir, 'mag_offset.png'), format='png', dpi=700)

In [ ]:
plt.figure(figsize=(6, 4))
plt.plot(detections['sim_mag_zps'], detections['MAG_ISO_zps'], 'b.', label='Zackay')
plt.plot(detections['sim_mag_ois'], detections['MAG_ISO_ois'], 'r.', label='Bramich')
plt.ylabel('Mag Aper')
plt.xlabel('Simulated Mag')
plt.title('Simulated Mag vs Detected Aperture Mag')
plt.legend(loc='best')
plt.savefig(os.path.join(plot_dir, 'aper_vs_simulated_mag.png'), format='png', dpi=700)

In [ ]:
print 'Mean magniutde difference for Zackay is {}'.format(detections.MAG_OFFSET_zps.mean())
print 'Mean magniutde difference for Bramich is {}'.format(detections.MAG_OFFSET_ois.mean())

In [ ]:
dt_ois['mag'] = dt_ois['MAG_ISO'] + detections.MAG_OFFSET_ois.mean()
dt_zps['mag'] = dt_zps['MAG_ISO'] + detections.MAG_OFFSET_zps.mean()

In [ ]:
plt.plot(detections['sim_mag_zps'], 
         detections['MAG_ISO_zps'] + detections.MAG_OFFSET_zps.mean(),
         'b.', label='Zackay', alpha=0.3)
plt.plot(detections['sim_mag_ois'],
         detections['MAG_ISO_ois'] + detections.MAG_OFFSET_ois.mean(),
         'r.', label='Bramich', alpha=0.3)
plt.ylabel('Mag Aper')
plt.xlabel('Simulated Mag')
plt.title('Simulated Mag vs Detected Aperture Mag')
plt.legend(loc='best')

plt.savefig(os.path.join(plot_dir, 'corrected_aper_vs_simulated_mag.png'), format='png', dpi=900)

In [ ]:
dt_ois['mu'] = dt_ois.mag/(dt_ois.A_IMAGE*dt_ois.B_IMAGE)
dt_zps['mu'] = dt_zps.mag/(dt_zps.A_IMAGE*dt_zps.B_IMAGE)

In [ ]:
bogus_b = dt_ois[dt_ois.IS_REAL==False]
reals_b = dt_ois[dt_ois.IS_REAL==True]

bogus_z = dt_zps[dt_zps.IS_REAL==False]
reals_z = dt_zps[dt_zps.IS_REAL==True]

## Luminosity function 

Bogus, Real, and FN luminosity functions.


In [ ]:
plt.figure(figsize=(10,9))

#magnitude bins
bins = np.arange(18, 25, 0.25)

plt.subplot(221)
plt.hist(reals_b.mag, alpha=0.5, bins=bins, normed=True, #shade=True, 
            cumulative=True, label='Bramich', color='red')
plt.hist(reals_z.mag, alpha=0.5, bins=bins, normed=True, #shade=True, 
            cumulative=True, label='Zackay', color='blue')

plt.xlim(18, 23.1)
plt.title('Cummulative Luminosity function Reals')
plt.ylabel(r'$\phi(m)dm$')
plt.legend(loc='best')
plt.xlabel(r'$m$')

plt.subplot(222)
plt.hist(bogus_b.mag, alpha=0.5, bins=bins, normed=True, #shade=True, 
            cumulative=True, label='Bramich', color='red')
plt.hist(bogus_z.mag, alpha=0.5, bins=bins, normed=True, #shade=True, 
            cumulative=True, label='Zackay', color='blue')

plt.xlim(20, 24.5)
plt.ylabel(r'$\phi(m)dm$')
plt.legend(loc='best')
plt.xlabel(r'$m$')
plt.title('Cummulative Luminosity function Bogus')

#signal to noise bins
bins = np.arange(0, 140, 3)

plt.subplot(223)
plt.hist(reals_b.FLUX_ISO/reals_b.FLUXERR_ISO,  bins=bins, normed=True, #shade=True, 
            label='Bramich', color='red', cumulative=-1, alpha=0.4)
plt.hist(reals_z.FLUX_ISO/reals_z.FLUXERR_ISO,  bins=bins, normed=True, #shade=True, 
         label='Zackay', color='blue', cumulative=-1, alpha=0.4)

plt.xlim(110, 0)
plt.title('Cummulative S/N function Reals')
plt.legend(loc='best')
plt.ylabel(r'$\phi$')
plt.xlabel('S/N')

plt.subplot(224)
plt.hist(bogus_b.FLUX_ISO/bogus_b.FLUXERR_ISO, bins=bins, normed=True, #shade=True, 
         label='Bramich', color='red', cumulative=-1, alpha=0.5)
plt.hist(bogus_z.FLUX_ISO/bogus_z.FLUXERR_ISO, bins=bins, normed=True, #shade=True, 
         label='Zackay', color='blue', cumulative=-1, alpha=0.5)

plt.xlim(70, 0)
plt.title('Cummulative S/N function Bogus')
plt.ylabel(r'$\phi$')
plt.legend(loc='best')
plt.xlabel('S/N')

plt.tight_layout()
#plt.show()

plt.savefig(os.path.join(plot_dir, 'luminosities_functions.png'), format='png', dpi=700)

## What about the FN?

We should check on the lost objects.

### Luminosity function for Undetected

In [ ]:
bins = np.arange(18, 24, 0.1)

plt.figure()
# sns.kdeplot(und_z.app_mag, cumulative=True, shade=True, color='blue', label='Zackay')
# sns.kdeplot(und_b.app_mag, cumulative=True, shade=True, color='red', label='Bramich')
plt.hist(und_z.app_mag, cumulative=True, bins=bins,
         #histtype='stepfilled', 
         color='blue', label='Zackay', alpha=0.5)
plt.hist(und_b.app_mag, cumulative=True, bins=bins, 
         #histtype='stepfilled', 
         color='red', label='Bramich', alpha=0.5)


plt.legend(loc='upper left')
plt.ylabel(r'$\int_{-\infty}^{m}\phi(m\prime)dm\prime$')
plt.xlabel(r'$m$')
plt.title('Cummulative Luminosity Function of False Negatives')
plt.xlim(21., 23.1)
#plt.show()

plt.savefig(os.path.join(plot_dir, 'lum_function_FNegatives.png'), format='png', dpi=700)

### Are there coincident objects?

In [ ]:
und_b.columns

In [ ]:
und_z.columns

In [ ]:
und_mix = pd.merge(left=und_b, right=und_z, on='simulated_id', suffixes=('_b', '_z'), how='outer', indicator='row_from')

In [ ]:
und_mix.columns

In [ ]:
magnitudes_und = pd.DataFrame({'mag_both' : und_mix[und_mix.row_from=='both'].app_mag_b,
                               'mag_onlyb': und_mix[und_mix.row_from=='left_only'].app_mag_b,
                               'mag_onlyz': und_mix[und_mix.row_from=='right_only'].app_mag_z})

In [ ]:
magnitudes_und.plot.hist(stacked=True, bins=15, log=True)

In [ ]:
bins = np.arange(19, 23., 0.25)
plt.hist(und_mix.app_mag_b[und_mix.row_from=='both'], bins=bins, log=True)
plt.title('False Positives in common')
plt.xlabel('mag')
plt.xlim(19, 23.)
plt.show()

In [ ]:
plt.figure(figsize=(14,5))
plt.subplot(121)
plt.xlabel
plt.hexbin(und_z.x, und_z.y, cmap='viridis')
plt.colorbar()
plt.subplot(122)
plt.hexbin(und_b.x, und_b.y, cmap='viridis')
plt.colorbar()

In [ ]:
plt.figure(figsize=(8, 4))
plt.subplot(121)
plt.title('Reals S/N vs mag')
plt.scatter(reals_b.mag, 
         np.log10(reals_b.FLUX_ISO/reals_b.FLUXERR_ISO),
         s=np.log10(reals_b.FLUXERR_ISO), linewidths=0.001, 
        c='r', marker='.', alpha=0.3, label='Bramich')
plt.scatter(reals_z.mag, 
         np.log10(reals_z.FLUX_ISO/reals_z.FLUXERR_ISO), 
         s=np.log10(reals_z.FLUXERR_ISO), linewidths=0.001,
         c='b', marker='.', alpha=0.3, label='Zackay')
plt.xlim(18,25)
#plt.ylim(0, 220)
plt.xlabel(r'$mag$')
plt.ylabel(r'$log(S/N)$')
plt.legend(loc='best')
plt.subplot(122)
plt.title('Bogus S/N vs mag')
plt.scatter(bogus_b.mag,
            np.log10(bogus_b.FLUX_ISO/bogus_b.FLUXERR_ISO),
            s=np.log10(bogus_b.FLUXERR_ISO), linewidths=0.001,
            c='r', marker='.', alpha=0.3, label='Bramich') 
plt.scatter(bogus_z.mag,
            np.log10(bogus_z.FLUX_ISO/bogus_z.FLUXERR_ISO),
            s=np.log10(bogus_z.FLUXERR_ISO), linewidths=0.001,
            c='b', marker='.', alpha=0.3, label='Zackay')
plt.xlim(18,25)
#plt.ylim(0, 220)
plt.xlabel(r'$mag$')
plt.ylabel(r'$log(S/N)$')
plt.legend(loc='best')
plt.tight_layout()
plt.savefig(os.path.join(plot_dir, 'signal_to_noise_vs_mag.png'), format='png', dpi=700)

In [ ]:
low_cstar_b = reals_b.CLASS_STAR < 0.005
low_cstar_z = reals_z.CLASS_STAR < 0.005

plt.figure(figsize=(8, 4))
plt.subplot(121)
plt.title('Reals S/N vs mag')
plt.scatter(reals_b.mag[low_cstar_b], 
         np.log10(reals_b.FLUX_ISO/reals_b.FLUXERR_ISO)[low_cstar_b],
         s=np.log10(reals_b.FLUXERR_ISO)[low_cstar_b], linewidths=0.001, 
        c='r', marker='.', alpha=0.3, label='Bramich')
plt.scatter(reals_z.mag[low_cstar_z], 
         np.log10(reals_z.FLUX_ISO/reals_z.FLUXERR_ISO)[low_cstar_z], 
         s=np.log10(reals_z.FLUXERR_ISO)[low_cstar_z], linewidths=0.001,
         c='b', marker='.', alpha=0.3, label='Zackay')
plt.xlim(18,25)
#plt.ylim(0, 220)
plt.xlabel(r'$mag$')
plt.ylabel(r'$log(S/N)$')
plt.legend(loc='best')


plt.subplot(122)
plt.title('Bogus S/N vs mag')
plt.scatter(bogus_b.mag,
            np.log10(bogus_b.FLUX_ISO/bogus_b.FLUXERR_ISO),
            s=np.log10(bogus_b.FLUXERR_ISO), linewidths=0.001,
            c='r', marker='.', alpha=0.3, label='Bramich') 
plt.scatter(bogus_z.mag,
            np.log10(bogus_z.FLUX_ISO/bogus_z.FLUXERR_ISO),
            s=np.log10(bogus_z.FLUXERR_ISO), linewidths=0.001,
            c='b', marker='.', alpha=0.3, label='Zackay')
plt.xlim(18,25)
#plt.ylim(0, 220)
plt.xlabel(r'$mag$')
plt.ylabel(r'$log(S/N)$')
plt.legend(loc='best')
plt.tight_layout()
plt.savefig(os.path.join(plot_dir, 'signal_to_noise_vs_mag.png'), format='png', dpi=700)

Need to calculate mean and dispersion of log10(S/N) on magnitude bins.

In [ ]:
def binning(data, bins, step):
    meanlogsn = []
    stdvlogsn = []
    
    for abin in bins:
        fdata = data[(data.mag < abin + step) * (data.mag >= abin)]
        meanlogsn.append(np.mean(np.log10(fdata.FLUX_ISO/fdata.FLUXERR_ISO)))
        stdvlogsn.append(np.std( np.log10(fdata.FLUX_ISO/fdata.FLUXERR_ISO)))
    return meanlogsn, stdvlogsn

In [ ]:
step = 0.3
bins = np.arange(19.6, 24.4, step)

plt.figure(figsize=(10, 5))
plt.subplot(121)
plt.title('Reals S/N vs mag')
means, stds = binning(reals_b, bins, step)
plt.errorbar(x=bins + step/2., 
             y=means,
             yerr=stds, 
             #linewidths=0.001, 
             c='r', marker='.', alpha=1, label='Bramich')

means, stds = binning(reals_z, bins, step)
plt.errorbar(x=bins + step/2., 
             y=means,
             yerr=stds, 
             c='b', marker='.', alpha=1, label='Zackay')
plt.xlim(19,25)
#plt.ylim(0, 220)
plt.xlabel(r'$mag$')
plt.ylabel(r'$log(S/N)$')
plt.legend(loc='best')

plt.subplot(122)
plt.title('Bogus S/N vs mag')
means, stds = binning(bogus_b, bins, step)
plt.errorbar(x=bins + step/2., 
             y=means,
             yerr=stds, 
            #linewidths=0.001,
            c='r', marker='.', alpha=1, label='Bramich') 
means, stds = binning(bogus_z, bins, step)
plt.errorbar(x=bins + step/2., 
             y=means,
             yerr=stds, 
             #linewidths=0.001,
             c='b', marker='.', alpha=1, label='Zackay')
plt.xlim(19,25)
#plt.ylim(0, 220)
plt.xlabel(r'$mag$')
plt.ylabel(r'$log(S/N)$')
plt.legend(loc='best')
plt.tight_layout()
plt.savefig(os.path.join(plot_dir, 'signal_to_noise_vs_mag_binned.png'), format='png', dpi=700)

In [ ]:
bins=np.arange(-2.5, 0., 0.1)
plt.figure(figsize=(8, 4))
plt.subplot(121)
plt.title('Reals')
plt.hist(np.log10(reals_b.MAGERR_ISO[reals_b.MAG_ISO < 30]), bins=bins,
         log=True, alpha=0.5, color='red', label='Bramich')
plt.hist(np.log10(reals_z.MAGERR_ISO[reals_z.MAG_ISO < 30]), bins=bins,
         log=True, alpha=0.5, color='blue', label='Zackay')
plt.xlabel(r'$Log(\epsilon)$')
plt.xlim(-2.5, 0.)
plt.ylim(0, 10000)
plt.legend(loc='best')

plt.subplot(122)
plt.title('Bogus')
plt.hist(np.log10(bogus_b.MAGERR_ISO[bogus_b.MAG_ISO < 30]), bins=bins,
         log=True, alpha=0.5, color='red', label='Bramich')
plt.hist(np.log10(bogus_z.MAGERR_ISO[bogus_z.MAG_ISO < 30]), bins=bins,
         log=True, alpha=0.5, color='blue', label='Zackay')
plt.xlabel(r'$Log(\epsilon)$')
plt.xlim(-2.5, 0.)
plt.ylim(0, 10000)
plt.legend(loc='best')

plt.tight_layout()
plt.savefig(os.path.join(plot_dir, 'mag_errors.png'), format='png', dpi=700)

In [ ]:
bins = np.arange(0, 1., 0.1)
plt.figure(figsize=(8, 4))
plt.subplot(121)
plt.title('Reals')
plt.hist(reals_b.CLASS_STAR, log=True, bins=bins, alpha=0.5, color='red', label='Bramich')
plt.hist(reals_z.CLASS_STAR, log=True, bins=bins, alpha=0.5, color='blue', label='Zackay')
plt.xlabel(r'Class Star')
plt.legend(loc='best')
plt.subplot(122)
plt.title('Bogus')
plt.hist(bogus_b.CLASS_STAR, log=True, bins=bins, alpha=0.5, color='red', label='Bramich')
plt.hist(bogus_z.CLASS_STAR, log=True, bins=bins, alpha=0.5, color='blue', label='Zackay')
plt.xlabel(r'Class Star')
plt.legend(loc='best')

plt.savefig(os.path.join(plot_dir, 'class_star_hist.png'), format='png', dpi=700)

In [ ]:
bins = np.arange(0, 100, 10)

plt.figure(figsize=(8, 4))
plt.subplot(121)
plt.title('Reals')
plt.hist(reals_b.mu, bins=bins, log=True, alpha=0.5, color='red', label='Bramich')
plt.hist(reals_z.mu, bins=bins, log=True, alpha=0.5, color='blue', label='Zackay')
plt.xlabel(r'$\mu$')
plt.legend(loc='best')
plt.subplot(122)
plt.title('Bogus')
plt.hist(bogus_b.mu, bins=bins, log=True, alpha=0.5, color='red', label='Bramich')
plt.hist(bogus_z.mu, bins=bins, log=True, alpha=0.5, color='blue', label='Zackay')
plt.xlabel(r'$\mu$')
plt.legend(loc='best')

plt.savefig(os.path.join(plot_dir, 'mu_hist.png'), format='png', dpi=700)

In [ ]:
bins = np.arange(0, 28, 2)

plt.figure(figsize=(8, 4))
plt.subplot(121)
plt.title('Reals')
plt.hist(reals_b.FLAGS, bins=bins, log=True, alpha=0.5, color='red', label='Bramich')
plt.hist(reals_z.FLAGS, bins=bins, log=True, alpha=0.5, color='blue', label='Zackay')
plt.xlabel(r'FLAGS')
plt.legend(loc='best')
plt.subplot(122)
plt.title('Bogus')
plt.hist(bogus_b.FLAGS, bins=bins, log=True, alpha=0.5, color='red', label='Bramich')
plt.hist(bogus_z.FLAGS, bins=bins, log=True, alpha=0.5, color='blue', label='Zackay')
plt.xlabel(r'FLAGS')
plt.legend(loc='best')

plt.savefig(os.path.join(plot_dir, 'flags_hist.png'), format='png', dpi=700)

In [ ]:
plt.figure(figsize=(8, 4))
plt.subplot(121)
plt.title('Reals')
plt.scatter(reals_b.ELONGATION, reals_b.ELLIPTICITY, s=reals_b.mu,
            alpha=0.5, color='red', label='Bramich')
plt.scatter(reals_z.ELONGATION, reals_z.ELLIPTICITY, s=reals_z.mu,
            alpha=0.5, color='blue', label='Zackay')
plt.xlabel(r'Elongation')
plt.ylabel(r'Ellipticity')
plt.legend(loc='best')
# plt.xlim(0, 16)
# plt.ylim(-0.1, 1.3)

plt.subplot(122)
plt.title('Bogus')
plt.scatter(bogus_b.ELONGATION, bogus_b.ELLIPTICITY, s=bogus_b.mag**2,
            alpha=0.3, color='red', label='Bramich')
plt.scatter(bogus_z.ELONGATION, bogus_z.ELLIPTICITY, s=bogus_z.mag**2,
            alpha=0.3, color='blue', label='Zackay')
plt.xlabel(r'Elongation')
plt.ylabel(r'Ellipticity')
plt.legend(loc='best')

plt.savefig(os.path.join(plot_dir, 'ellipticty_vs_elongation.png'), format='png', dpi=700)

In [ ]:
plt.figure(figsize=(8, 4))
plt.subplot(121)
plt.title('Reals')
plt.plot(reals_b.CLASS_STAR, reals_b.mu, '.',
            alpha=0.5, color='red', label='Bramich')
plt.plot(reals_z.CLASS_STAR, reals_z.mu, '.', 
            alpha=0.5, color='blue', label='Zackay')

# plt.xlim(0, 16)
# plt.ylim(-0.1, 1.3)
plt.loglog()
plt.xlabel(r'CLASS_STAR')
plt.ylabel(r'$\mu$')
plt.legend(loc='best')

plt.subplot(122)
plt.title('Bogus')
plt.plot(bogus_b.CLASS_STAR, bogus_b.mu, '.',
            alpha=0.5, color='red', label='Bramich')
plt.plot(bogus_z.CLASS_STAR, bogus_z.mu, '.', 
            alpha=0.5, color='blue', label='Zackay')
plt.ylim(1, 100.)
plt.loglog()
plt.xlabel(r'CLASS_STAR')
plt.ylabel(r'$\mu$')
plt.legend(loc='best')


## Simulation parameters

We need data on simulation for every object, specially undetected (there are a lot of these)

In [ ]:
sim_pars = pd.read_sql_query("""SELECT * FROM Images""", engine)

Now we will merge our simulation parameters on undetected objects:

In [ ]:
und_b = pd.merge(left=und_b, right=sim_pars, left_on='image_id', right_on='id', how='left')
und_z = pd.merge(left=und_z, right=sim_pars, left_on='image_id', right_on='id', how='left')

In [ ]:
print ' Columns of sim_pars:\n', sim_pars.columns
print '\n Columns of und_b:\n', und_b.columns
print '\n Columns of und_z:\n', und_z.columns
print '\n Columns of simulated:\n', simulated.columns

In [ ]:
plt.figure(figsize=(9, 6))
plt.subplot(2, 3, 1)
plt.hist(und_z['refseeing_fwhm'], cumulative=False, histtype='bar', alpha=0.5 , color='blue', normed=True)
plt.hist(und_b['refseeing_fwhm'], cumulative=False, histtype='bar', alpha=0.5, color='red', normed=True)
plt.xlabel('seeing fwhm')
plt.title('REF Seeing for undetected')

plt.subplot(2, 3, 2)
plt.hist(und_z['refstarcount_slope'], cumulative=False, histtype='bar', alpha=0.5 , color='blue', normed=True)
plt.hist(und_b['refstarcount_slope'], cumulative=False, histtype='bar', alpha=0.5, color='red', normed=True)
plt.xlabel('slope')
plt.title('REF Starcount slope for undetected')

plt.subplot(2, 3, 3)
plt.hist(np.log10(und_z['refstarcount_zp']), cumulative=False, histtype='bar', 
         alpha=0.5 , color='blue', normed=True)
plt.hist(np.log10(und_b['refstarcount_zp']), cumulative=False, histtype='bar', 
         alpha=0.5, color='red', normed=True)
plt.xlabel('zp')
plt.title('REF Starcount ZP for undetected')

plt.subplot(2, 3, 4)
plt.hist(und_z['r_scales'], cumulative=False, histtype='bar', alpha=0.5, color='blue', log=True, normed=True)
plt.hist(und_b['r_scales'], cumulative=False, histtype='bar', alpha=0.5, color='red', log=True, normed=True)
plt.xlabel(r'$r$')
plt.title('Distance to host')

plt.subplot(2, 3, 5)
plt.hist(und_z['gx_mag'], cumulative=False, histtype='bar', alpha=0.5, color='blue', log=True, normed=True)
plt.hist(und_b['gx_mag'], cumulative=False, histtype='bar', alpha=0.5, color='red', log=True, normed=True)
plt.xlabel(r'gx_mag')
plt.title('Host Galaxy magnitud')


plt.tight_layout()
plt.show()

In [ ]:
reals_b = pd.merge(left=reals_b, right=sim_pars, left_on='image_id', right_on='id', how='left')
reals_z = pd.merge(left=reals_z, right=sim_pars, left_on='image_id', right_on='id', how='left')

In [ ]:
plt.figure(figsize=(9, 6))
plt.subplot(2, 3, 1)
plt.hist(reals_z['refseeing_fwhm'], cumulative=False, histtype='bar',
         alpha=0.5 , color='blue', normed=True)
plt.hist(reals_b['refseeing_fwhm'], cumulative=False, histtype='bar',
         alpha=0.5, color='red', normed=True)
plt.xlabel('seeing fwhm')
plt.title('REF Seeing for Reals')

plt.subplot(2, 3, 2)
plt.hist(reals_z['refstarcount_slope'], cumulative=False, histtype='bar',
         alpha=0.5 , color='blue', normed=True)
plt.hist(reals_b['refstarcount_slope'], cumulative=False, histtype='bar',
         alpha=0.5, color='red', normed=True)
plt.xlabel('slope')
plt.title('REF Starcount slope for Reals')

plt.subplot(2, 3, 3)
plt.hist(np.log10(reals_z['refstarcount_zp']), cumulative=False, histtype='bar', 
         alpha=0.5 , color='blue', normed=True)
plt.hist(np.log10(reals_b['refstarcount_zp']), cumulative=False, histtype='bar', 
         alpha=0.5, color='red', normed=True)
plt.xlabel('zp')
plt.title('REF Starcount ZP for Reals')

plt.subplot(2, 3, 4)
plt.hist(reals_z['r_scales'], cumulative=False, histtype='bar',
         alpha=0.5, color='blue', log=True, normed=True)
plt.hist(reals_b['r_scales'], cumulative=False, histtype='bar',
         alpha=0.5, color='red', log=True, normed=True)
plt.xlabel(r'$r$')
plt.title('Distance to host for Reals')

plt.subplot(2, 3, 5)
plt.hist(reals_z['gx_mag'], cumulative=False, histtype='bar',
         alpha=0.5, color='blue', log=True, normed=True)
plt.hist(reals_b['gx_mag'], cumulative=False, histtype='bar',
         alpha=0.5, color='red', log=True, normed=True)
plt.xlabel(r'gx_mag')
plt.title('Host Galaxy magnitud for Reals')


plt.tight_layout()
plt.show()

In [ ]:
print bogus_b.columns
print bogus_z.columns
print sim_pars.columns

## Checking on the outliers

In [ ]:
from astropy.nddata.utils import extract_array
from astropy.io import fits


def get_patch_bramich(x, y, image_id):
    """Gets data from a row of a table, from a bramich subtracted source, and uses
    its coordinates to extract a patch from the image.
    """
    size = (14, 14)
    
    path = os.path.join('/home/bruno/Data/RBpipeline/images', 
                        'img'+str(image_id).zfill(5))
    filepath = os.path.join(path, 'diff_ois.fits')
    
    pixdata = fits.getdata(filepath)
    
    patch = extract_array(pixdata, size, (x, y))
    
    return patch
    
    
def get_patch_zackay(x, y, image_id):
    """Gets data from a row of a table, from a bramich subtracted source, and uses
    its coordinates to extract a patch from the image.
    """
    size = (14, 14)
    
    path = os.path.join('/home/bruno/Data/RBpipeline/images', 
                        'img'+str(image_id).zfill(5))
    filepath = os.path.join(path, 'diff.fits')
    
    pixdata = fits.getdata(filepath)
    
    patch = extract_array(pixdata, size, (x, y))
    
    return patch
    

In [ ]:
outliers_b = bogus_b[bogus_b.CLASS_STAR > .97]

In [ ]:
outliers_b

In [ ]:
plt.figure(figsize=( 6, 2*len(outliers_b)))
for i in range(len(outliers_b)):
    patch = get_patch_bramich(outliers_b[i:i+1]['Y_IMAGE'], 
                              outliers_b[i:i+1]['X_IMAGE'], 
                              int(outliers_b[i:i+1]['image_id']))
    plt.subplot(len(outliers_b), 1 , i+1)
    plt.grid()
    plt.imshow(patch, cmap='viridis', interpolation='none')

In [ ]:
outliers_z = bogus_z[bogus_z.ELONGATION > 8]

In [ ]:
outliers_z

In [ ]:
plt.figure(figsize=(6*len(outliers_z), 12))
for i in range(len(outliers_z)):
    patch = get_patch_zackay(outliers_z[i:i+1]['Y_IMAGE'], 
                              outliers_z[i:i+1]['X_IMAGE'], 
                              int(outliers_z[i:i+1]['image_id']))
    plt.subplot(1, len(outliers_z), i+1)
    plt.grid()
    plt.imshow(patch, cmap='viridis', interpolation='none')